модель пс
модель бс
тест что инвариантны
тест что воспроизводят либраскаловскую штуковину

In [1]:
'''!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz
!gunzip -k methane.extxyz.gz'''

'!wget "https://archive.materialscloud.org/record/file?file_id=b612d8e3-58af-4374-96ba-b3551ac5d2f4&filename=methane.extxyz.gz&record_id=528" -O methane.extxyz.gz\n!gunzip -k methane.extxyz.gz'

In [2]:
import numpy as np
from rascal.representations import SphericalExpansion as SPH
import ase.io
import tqdm
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge
import torch
torch.set_num_threads(1)
import torch.nn
from code_pytorch import *
import time
import numpy as np
from sympy.physics.wigner import wigner_3j
from sympy.physics.quantum.spin import Rotation as rot
from rascal.representations import SphericalInvariants as SOAP
from torch2trt import torch2trt

In [3]:
class Powerspectrum(torch.nn.Module):
    def __init__(self, clebsch):
        super(Powerspectrum, self).__init__()
        self.first = ClebschCombining(clebsch, 0)       
            
    def forward(self, X):
        ps_invariants = self.first(X, X)       
        return ps_invariants
    
class Bispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Bispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_invariants = self.second(ps_covariants, X)
        return bs_invariants
    
def convert_to_torch(coefficients, device):
    result = {}
    for lambd in range(coefficients.shape[2]):
        result[lambd] = torch.FloatTensor(coefficients[:, :, lambd, : 2 * lambd + 1]).to(device)
        result[lambd] = result[lambd].transpose(0, -1)
    return result

In [4]:
LAMBDA_MAX = 5

HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 5,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.3,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}

HYPERS_PS = copy.deepcopy(HYPERS)
HYPERS_PS['soap_type'] = 'PowerSpectrum'

HYPERS_BS = copy.deepcopy(HYPERS)
HYPERS_BS['soap_type'] = 'BiSpectrum'
HYPERS_BS['inversion_symmetry'] = False

In [5]:
def get_rascal_ps(structures):
    #nice.rascal_coefficients.process_structures(structures)
    soap_ps = SOAP(**HYPERS_PS)   
    ps = soap_ps.transform(structures).get_features(soap_ps)
    return ps

def get_rascal_bs(structures):
    soap_bs = SOAP(**HYPERS_BS)    
    bs = soap_bs.transform(structures).get_features(soap_bs)
    return bs

def get_torch_invariants(structures, device, model):
    all_species = get_all_species(structures)
    coefficients = get_spherical_expansion(structures, HYPERS,
                                             all_species)    
    
    result = {}
    for specie in all_species:
        result[specie] = model(convert_to_torch(coefficients[specie], device))
        
    
    return result

In [6]:
structures = ase.io.read('methane.extxyz', index='0:300')
structures = nice.rascal_coefficients.process_structures(structures) #put all inside unit cell for rascal
clebsch = nice.clebsch_gordan.ClebschGordan(LAMBDA_MAX)

In [7]:
begin = time.time()
_ = get_rascal_ps(structures)
print(time.time() - begin)

0.061212778091430664


In [8]:
model_ps = Powerspectrum(clebsch.precomputed_)
begin = time.time()
_ = get_torch_invariants(structures, 'cpu', model_ps)
print(time.time() - begin)

100%|██████████| 2/2 [00:00<00:00, 563.33it/s]

0.3246273994445801


In [9]:
model_ps = model_ps.cuda()
begin = time.time()
_ = get_torch_invariants(structures, 'cuda', model_ps)
print(time.time() - begin)

100%|██████████| 2/2 [00:00<00:00, 820.32it/s]

0.39711785316467285


In [10]:
begin = time.time()
_ = get_rascal_bs(structures)
print(time.time() - begin)

102.4657506942749


In [11]:
model_bs = Bispectrum(clebsch.precomputed_, LAMBDA_MAX)
begin = time.time()
_ = get_torch_invariants(structures, 'cpu', model_bs)
print(time.time() - begin)

100%|██████████| 2/2 [00:00<00:00, 714.05it/s]


26.96322751045227


In [12]:
model_bs = model_bs.cuda()
begin = time.time()
_ = get_torch_invariants(structures, 'cuda', model_bs)
print(time.time() - begin)

100%|██████████| 2/2 [00:00<00:00, 719.74it/s]


2.9388179779052734
